In [1]:
# from keras.preprocessing import image
import keras.utils as image
import os
import matplotlib.pyplot as plt
from random import randint

NUM_INSTRUMENTS = 11

instruments = ["cel", "cla", "flu", "gac", "gel", "org", "pia", "sax", "tru", "vio", "voi"]

inst_dict = {}
for i in range(len(instruments)): 
    inst_dict[instruments[i]] = i

def load_images_from_path(path, x_array, y_array, index, label = None, path_label = None):
    images = []
    labels = []
    temp_label = [0 for _ in range(NUM_INSTRUMENTS)]
    if not path_label:
        temp_label[label] = 1     
        
        
    for file in os.listdir(path):
        x_array[index] = image.img_to_array(image.load_img(os.path.join(path, file), target_size=(224, 224, 3)))
        
        # images.append(image.img_to_array(image.load_img(os.path.join(path, file), target_size=(224, 224, 3))))
        if path_label:
            temp_label = [False for _ in range(NUM_INSTRUMENTS)]
            with open("{}/{}.txt".format(path_label, file[:-4]), "r") as f:
                for inst in f:
                    inst = inst.replace('\n', '').replace('\t', '')
                    temp_label[inst_dict[inst]] = 1
        # labels.append((temp_label))
        y_array[index] = temp_label
        index += 1
        
        
    # print(len(images), len(os.listdir(path)), len(images[0]))
    # return images, labels
    return index

def show_images(images):
    fig, axes = plt.subplots(1, 8, figsize=(20, 20), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i] / 255)
        

In [2]:
from tensorflow.keras.utils import to_categorical
import numpy as np
len_train = 0
for i in range(len(instruments)):
    len_train += len(os.listdir('Spectrograms/{}'.format(instruments[i])))
print(len_train)

x = np.empty(shape = (len_train, 224, 224, 3))
y = np.empty(shape = (len_train, NUM_INSTRUMENTS))
index = 0
for i in range(len(instruments)):
    index = load_images_from_path(path = 'Spectrograms/{}'.format(instruments[i]), label = i, x_array = x, y_array = y, index = index)
    # images, labels = load_images_from_path(path = 'Spectrograms/{}'.format(instruments[i]), label = i)
    # show_images(images)
    
    # np.append(x, images)
    # np.append(y, labels)

y_enc = to_categorical(y)
print(len(x))
print(len(y))

7210
7210
7210


In [5]:
x_test = np.empty(shape = (len(os.listdir('Spectrograms/TEST')), 224, 224, 3))
y_test = np.empty(shape = (len(os.listdir('Spectrograms/TEST')), NUM_INSTRUMENTS))
y_test_enc = to_categorical(y_test)
load_images_from_path(path = 'Spectrograms/TEST', path_label = 'Dataset/IRMAS_Validation_Data', x_array = x_test, y_array = y_test, index = 0)

2874

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense
import keras.backend
from keras.losses import losses_utils
import keras.losses


# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(11, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [4]:
import numpy as np
print(np.shape(x))
print(np.shape(y))
print(np.shape(x_test))
print(np.shape(y_test))

(7210, 224, 224, 3)
(7210, 11)
(2874, 224, 224, 3)
(2874, 11)


In [8]:
hist = model.fit(x, y, validation_data=(x_test, y_test), batch_size=32, epochs=20, shuffle = True, use_multiprocessing= True, workers = 4)

7210 7210 2874 2874
Epoch 1/20
226/226 [==============================] - 280s 1s/step - loss: 13.1073 - accuracy: 0.1773 - val_loss: 0.4419 - val_accuracy: 0.1343
Epoch 2/20
226/226 [==============================] - 265s 1s/step - loss: 0.2717 - accuracy: 0.2731 - val_loss: 0.4565 - val_accuracy: 0.1385
Epoch 3/20
226/226 [==============================] - 265s 1s/step - loss: 0.2465 - accuracy: 0.3603 - val_loss: 0.4885 - val_accuracy: 0.1447
Epoch 4/20
226/226 [==============================] - 262s 1s/step - loss: 0.2229 - accuracy: 0.4284 - val_loss: 0.4983 - val_accuracy: 0.2008
Epoch 5/20
226/226 [==============================] - 258s 1s/step - loss: 0.2062 - accuracy: 0.4899 - val_loss: 0.5930 - val_accuracy: 0.1340
Epoch 6/20
226/226 [==============================] - 257s 1s/step - loss: 0.1862 - accuracy: 0.5483 - val_loss: 0.6188 - val_accuracy: 0.1795
Epoch 7/20
226/226 [==============================] - 258s 1s/step - loss: 0.1732 - accuracy: 0.6000 - val_loss: 0.6954 -

In [ ]:
model.save('Model_final')